# NO2 Data Aggregation - Plots

Aggregates NO2 values by administrative boundaries and a regular hexagon grid.

To convert from mol/m<sup>2</sup> to molecules/cm<sup>2</sup>, multiply by 6.022140857e+19 (6.022140857 * 10<sup>19</sup>).

# Environment Setup

In [ ]:
# Load Notebook formatter
%load_ext nb_black
# %reload_ext nb_black

In [ ]:
# Import packages
import os
import glob
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import seaborn as sns
import pandas as pd
import geopandas as gpd
import rasterio as rio
import rasterstats as rs
import sentinel as stl

In [ ]:
# Set Options
# sns.set(font_scale=1.5, style="whitegrid")
sns.set(font_scale=1.5)
pd.set_option("display.max_columns", None)
pd.set_option("precision", 15)

In [ ]:
# Set working directory
os.chdir("..")
print(f"Working directory: {os.getcwd()}")

# User-Defined Variables

In [ ]:
# Set paths to South Korea shapefiles at levels 0, 1, and 2, and hexagon grid
south_korea_level_0_path = os.path.join(
    "02-raw-data", "vector", "south-korea", "gadm36_south_korea.shp"
)

south_korea_level_1_path = os.path.join(
    "02-raw-data", "vector", "south-korea", "gadm36_south_korea_level_1.shp"
)

south_korea_level_2_path = os.path.join(
    "02-raw-data", "vector", "south-korea", "gadm36_south_korea_level_2.shp"
)

south_korea_hexagon_grid_path = os.path.join(
    "03-processed-data",
    "vector",
    "south-korea",
    "south_korea_hexagon_grid.shp",
)

In [ ]:
# Set paths to aggregated data shapfiles
aggregate_level_1_path = os.path.join(
    "03-processed-data",
    "vector",
    "south-korea",
    "aggregate_level_1_jul_2018_jul_2020.shp",
)

aggregate_level_2_path = os.path.join(
    "03-processed-data",
    "vector",
    "south-korea",
    "aggregate_level_2_jul_2018_jul_2020.shp",
)

aggregate_hexagon_path = os.path.join(
    "03-processed-data",
    "vector",
    "south-korea",
    "aggregate_hexagon_grid_jul_2018_jul_2020.shp",
)

# Data Acquisition and Preprocessing

In [ ]:
# Get geodataframes with geometry only (for plotting)
south_korea_level_0_geometry = stl.get_geometry(south_korea_level_0_path)
south_korea_level_1_geometry = stl.get_geometry(south_korea_level_1_path)
south_korea_level_2_geometry = stl.get_geometry(south_korea_level_2_path)
south_korea_hexagon_grid_geometry = stl.get_geometry(
    south_korea_hexagon_grid_path
)

In [ ]:
# Read aggregated shapefiles into geodataframes
south_korea_level_1_gdf = gpd.read_file(aggregate_level_1_path)
south_korea_level_2_gdf = gpd.read_file(aggregate_level_2_path)
hexagon_grid_gdf = gpd.read_file(aggregate_hexagon_path)

In [ ]:
# Restore original column names (shorted by export to shapefile)
level_1_gdf = south_korea_level_1_gdf.rename(
    columns={
        # Magnitude
        "change_jul": "change_jul_2018_2019_magnitude",
        "change_aug": "change_aug_2018_2019_magnitude",
        "change_sep": "change_sep_2018_2019_magnitude",
        "change_oct": "change_oct_2018_2019_magnitude",
        "change_nov": "change_nov_2018_2019_magnitude",
        "change_dec": "change_dec_2018_2019_magnitude",
        "change_jan": "change_jan_2019_2020_magnitude",
        "change_feb": "change_feb_2019_2020_magnitude",
        "change_mar": "change_mar_2019_2020_magnitude",
        "change_apr": "change_apr_2019_2020_magnitude",
        "change_may": "change_may_2019_2020_magnitude",
        "change_jun": "change_jun_2019_2020_magnitude",
        "change_j_1": "change_jul_2019_2020_magnitude",
        # Percent
        "change_j_2": "change_jul_2018_2019_percent",
        "change_a_1": "change_aug_2018_2019_percent",
        "change_s_1": "change_sep_2018_2019_percent",
        "change_o_1": "change_oct_2018_2019_percent",
        "change_n_1": "change_nov_2018_2019_percent",
        "change_d_1": "change_dec_2018_2019_percent",
        "change_j_3": "change_jan_2019_2020_percent",
        "change_f_1": "change_feb_2019_2020_percent",
        "change_m_1": "change_mar_2019_2020_percent",
        "change_a_2": "change_apr_2019_2020_percent",
        "change_m_2": "change_may_2019_2020_percent",
        "change_j_4": "change_jun_2019_2020_percent",
        "change_j_5": "change_jul_2019_2020_percent",
    }
)

level_2_gdf = south_korea_level_2_gdf.rename(
    columns={
        # Magnitude
        "change_jul": "change_jul_2018_2019_magnitude",
        "change_aug": "change_aug_2018_2019_magnitude",
        "change_sep": "change_sep_2018_2019_magnitude",
        "change_oct": "change_oct_2018_2019_magnitude",
        "change_nov": "change_nov_2018_2019_magnitude",
        "change_dec": "change_dec_2018_2019_magnitude",
        "change_jan": "change_jan_2019_2020_magnitude",
        "change_feb": "change_feb_2019_2020_magnitude",
        "change_mar": "change_mar_2019_2020_magnitude",
        "change_apr": "change_apr_2019_2020_magnitude",
        "change_may": "change_may_2019_2020_magnitude",
        "change_jun": "change_jun_2019_2020_magnitude",
        "change_j_1": "change_jul_2019_2020_magnitude",
        # Percent
        "change_j_2": "change_jul_2018_2019_percent",
        "change_a_1": "change_aug_2018_2019_percent",
        "change_s_1": "change_sep_2018_2019_percent",
        "change_o_1": "change_oct_2018_2019_percent",
        "change_n_1": "change_nov_2018_2019_percent",
        "change_d_1": "change_dec_2018_2019_percent",
        "change_j_3": "change_jan_2019_2020_percent",
        "change_f_1": "change_feb_2019_2020_percent",
        "change_m_1": "change_mar_2019_2020_percent",
        "change_a_2": "change_apr_2019_2020_percent",
        "change_m_2": "change_may_2019_2020_percent",
        "change_j_4": "change_jun_2019_2020_percent",
        "change_j_5": "change_jul_2019_2020_percent",
    }
)

hexagon_grid_gdf = hexagon_grid_gdf.rename(
    columns={
        # Magnitude
        "change_jul": "change_jul_2018_2019_magnitude",
        "change_aug": "change_aug_2018_2019_magnitude",
        "change_sep": "change_sep_2018_2019_magnitude",
        "change_oct": "change_oct_2018_2019_magnitude",
        "change_nov": "change_nov_2018_2019_magnitude",
        "change_dec": "change_dec_2018_2019_magnitude",
        "change_jan": "change_jan_2019_2020_magnitude",
        "change_feb": "change_feb_2019_2020_magnitude",
        "change_mar": "change_mar_2019_2020_magnitude",
        "change_apr": "change_apr_2019_2020_magnitude",
        "change_may": "change_may_2019_2020_magnitude",
        "change_jun": "change_jun_2019_2020_magnitude",
        "change_j_1": "change_jul_2019_2020_magnitude",
        # Percent
        "change_j_2": "change_jul_2018_2019_percent",
        "change_a_1": "change_aug_2018_2019_percent",
        "change_s_1": "change_sep_2018_2019_percent",
        "change_o_1": "change_oct_2018_2019_percent",
        "change_n_1": "change_nov_2018_2019_percent",
        "change_d_1": "change_dec_2018_2019_percent",
        "change_j_3": "change_jan_2019_2020_percent",
        "change_f_1": "change_feb_2019_2020_percent",
        "change_m_1": "change_mar_2019_2020_percent",
        "change_a_2": "change_apr_2019_2020_percent",
        "change_m_2": "change_may_2019_2020_percent",
        "change_j_4": "change_jun_2019_2020_percent",
        "change_j_5": "change_jul_2019_2020_percent",
    }
)

# Data Processing

# Data Post-Processing

# Data Visualization

## Level 1 Boundaries

In [ ]:
# Determine plot max
abs_max = abs(level_1_gdf["change_mar_2019_2020_magnitude"].max())
abs_min = abs(level_1_gdf["change_mar_2019_2020_magnitude"].min())
mag_vmax = abs_max if abs_max > abs_min else abs_min

# Plot magnitude and percent change
with plt.style.context("dark_background"):
    fig, ax = plt.subplots(1, 2, figsize=(20, 8))
    plt.suptitle(f"South Korea Nitrogen Dioxide Change, March 2019 to March 2020", 
                 size=24)
    
    # Set legend options
    divider_magnitude = make_axes_locatable(ax[0])
    divider_percent = make_axes_locatable(ax[1])
    cax_magnitude = divider_magnitude.append_axes("right", size="5%", pad=-0.25)
    cax_percent = divider_percent.append_axes("right", size="5%", pad=-0.25)

    # Magnitude change
    level_1_gdf.plot(
        column="change_mar_2019_2020_magnitude",
        ax=ax[0],
        legend=True,
        cax=cax_magnitude,
        vmax=mag_vmax,
        vmin=-mag_vmax,
        cmap="RdBu_r",
        linewidth=0.25,
        edgecolor="gray",
    )
#     south_korea_level_1_geometry.plot(
#         ax=ax[0], facecolor="None", edgecolor="red", linewidth=0.5)
        
    ax[0].set_title("Magnitude Change")
    ax[0].set_xlabel("Longitude (degrees)")
    ax[0].set_ylabel("Latitude (degrees)")

    # Percent change
    level_1_gdf.plot(
        column="change_mar_2019_2020_percent",
        ax=ax[1],
        legend=True,
        cax=cax_percent,
        cmap="RdBu_r",
        vmin=-100,
        vmax=100,
        linewidth=0.25,
        edgecolor="gray"
    )
#     south_korea_level_1_geometry.plot(
#         ax=ax[1], facecolor="None", edgecolor="red", linewidth=0.5,
#     )
    ax[1].set_title("Percent Change")
    ax[1].set_xlabel("Longitude (degrees)")
    ax[1].set_ylabel("Latitude (degrees)")
    
#     ax[1].legend(
#         shadow=True, edgecolor="white", fontsize=10, loc="lower right"
#     )
    
    fig.text(0.5, 0, f"Data Source: European Space Agency", ha="center", fontsize=16
        )


## Level 2 Boundaries

In [ ]:
# Determine plot max
abs_max = abs(level_2_gdf["change_mar_2019_2020_magnitude"].max())
abs_min = abs(level_2_gdf["change_mar_2019_2020_magnitude"].min())
mag_vmax = abs_max if abs_max > abs_min else abs_min

# Plot magnitude and percent change
with plt.style.context("dark_background"):
    fig, ax = plt.subplots(1, 2, figsize=(20, 8))
    plt.suptitle(f"South Korea Nitrogen Dioxide Change, March 2019 to March 2020", 
                 size=24)
    
    # Set legend options
    divider_magnitude = make_axes_locatable(ax[0])
    divider_percent = make_axes_locatable(ax[1])
    cax_magnitude = divider_magnitude.append_axes("right", size="5%", pad=-0.25)
    cax_percent = divider_percent.append_axes("right", size="5%", pad=-0.25)

    # Magnitude change
    level_2_gdf.plot(
        column="change_mar_2019_2020_magnitude",
        ax=ax[0],
        legend=True,
        cax=cax_magnitude,
        vmax=mag_vmax,
        vmin=-mag_vmax,
        cmap="RdBu_r",
        linewidth=0.25,
        edgecolor="gray"
#         color='black'
    )
#     south_korea_level_2_geometry.plot(
#         ax=ax[0], facecolor="None", edgecolor="black", linewidth=0.5)
        
    ax[0].set_title("Magnitude Change")
    ax[0].set_xlabel("Longitude (degrees)")
    ax[0].set_ylabel("Latitude (degrees)")

    # Percent change
    level_2_gdf.plot(
        column="change_mar_2019_2020_percent",
        ax=ax[1],
        legend=True,
        cax=cax_percent,
        cmap="RdBu_r",
        vmin=-100,
        vmax=100,
        linewidth=0.25,
        edgecolor="gray"
    )
#     south_korea_level_1_geometry.plot(
#         ax=ax[1], facecolor="None", edgecolor="red", linewidth=0.5,
#     )
    ax[1].set_title("Percent Change")
    ax[1].set_xlabel("Longitude (degrees)")
    ax[1].set_ylabel("Latitude (degrees)")
    
#     ax[1].legend(
#         shadow=True, edgecolor="white", fontsize=10, loc="lower right"
#     )
    
    fig.text(0.5, 0, f"Data Source: European Space Agency", ha="center", fontsize=16
        )


## Hexagon Grid

In [ ]:
# Determine plot max
abs_max = abs(hexagon_grid_gdf["change_mar_2019_2020_magnitude"].max())
abs_min = abs(hexagon_grid_gdf["change_mar_2019_2020_magnitude"].min())
mag_vmax = abs_max if abs_max > abs_min else abs_min

# Plot magnitude and percent change
with plt.style.context("dark_background"):
    fig, ax = plt.subplots(1, 2, figsize=(20, 8))
    plt.suptitle(
        f"South Korea Nitrogen Dioxide Change, March 2019 to March 2020", 
        size=24
    )
    
    # Set legend options
    divider_magnitude = make_axes_locatable(ax[0])
    divider_percent = make_axes_locatable(ax[1])
    cax_magnitude = divider_magnitude.append_axes("right", size="5%", pad=-0.25)
    cax_percent = divider_percent.append_axes("right", size="5%", pad=-0.25)

    # Magnitude change
    hexagon_grid_gdf.plot(
        column="change_mar_2019_2020_magnitude",
        ax=ax[0],
        legend=True,
        cax=cax_magnitude,
        vmax=mag_vmax,
        vmin=-mag_vmax,
        cmap="RdBu_r",
        linewidth=0.25,
#         edgecolor="gray"
    )
    south_korea_level_1_geometry.plot(
        ax=ax[0], facecolor="None", edgecolor="red", linewidth=0.5)
        
    ax[0].set_title("Magnitude Change")
    ax[0].set_xlabel("Longitude (degrees)")
    ax[0].set_ylabel("Latitude (degrees)")

    # Percent change
    hexagon_grid_gdf.plot(
        column="change_mar_2019_2020_percent",
        ax=ax[1],
        legend=True,
        cax=cax_percent,
        cmap="RdBu_r",
        vmin=-100,
        vmax=100,
        linewidth=0.25,
#         edgecolor="gray"
    )
    south_korea_level_1_geometry.plot(
        ax=ax[1], facecolor="None", edgecolor="red", linewidth=0.5,
    )
    ax[1].set_title("Percent Change")
    ax[1].set_xlabel("Longitude (degrees)")
    ax[1].set_ylabel("Latitude (degrees)")
    
#     ax[1].legend(
#         shadow=True, edgecolor="white", fontsize=10, loc="lower right"
#     )
    
    fig.text(0.5, 0, f"Data Source: European Space Agency", ha="center", fontsize=16
        )


# Data Export